<a href="https://colab.research.google.com/github/andreacangiani/NSPDE-ANA2024/blob/main/Python/CP7_worked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The FEniCS Project

The [FEniCS Project](https://colab.research.google.com/drive/1UX17QtYCpfLQhdu_Z8c3b3VlEh1Onjf8#scrollTo=u-kZtjlmAhjr&line=1&uniqifier=1) is an open-source software project aimed at creating an automated workflow for computational mathematical modelling based on the Finite Element Method (FEM).

The latest version of the FEniCS project, FEniCSx, consists of several building blocks:
* dolfinx is the FEM high performance C++ backend of FEniCSx, implementing structures such as meshes, function spaces and functions. DOLFINx also performs finite element assembly and mesh refinement algorithms. Finally, it interfaces to linear algebra solvers and data-structures, such as [PETSc](https://petsc.org/release/).
* UFL is a symbolic library providing a high-level form language for describing variational formulations with a high-level mathematical syntax
* FFCx is the form compiler of FEniCSx; given variational formulations written with UFL, it generates efficient C code.


As many other open-source software, FEniCS uses other packages while carrying out specific tasks of the FEM pipeline. A few notable dependencies of FEniCS are:

*  PETSc (and its Python wrapping petsc4py) for linear algebra solvers (and much more, such as nonlinear solvers and time stepping);
*    SLEPc (and its Python wrapping slepc4py) for solution of eigenvalue problems;
*    MPI for parallel computing;
*    ParMETIS and SCOTCH for mesh partitioning in parallel computing;
*    Gmsh for generation of complex meshes;
*    numpy for matrix/vector manipulation from Python;
*    viskex for plotting meshes and solutions.


More details can be found in the original tutorial:
* Hans Petter Langtangen, Anders Logg, *[Solving PDEs in Python: The FEniCS Tutorial I](https://link.springer.com/book/10.1007/978-3-319-52462-7)*, Simula SpringerBriefs on Computing, Springer, 2016. The codes are found [here](https://jorgensd.github.io/dolfinx-tutorial/).

This tutorial is a revised version of a tutorial by Francesco Ballarin (Università Cattolica del Sacro Cuore, Brescia), one of the developer of the FEniCS project. Credits go to him.

# FEniCS modules

In this tutorial we will explicitly use only a few libraries, namely numpy, petsc4py, UFL, dolfinx. However, all FEniCS software components (and many of the dependencies listed above) will be used under the hood

We start by importing all modules which we require.

We do this through the python try/except blocks:
* the try block lets you test a block of code for errors.
* the except block lets you handle the error.

**Note!** These libraries are in continuous development, so the following call may not work at a later time. For this, always check:

 https://fem-on-colab.github.io/


In [1]:
try:
    import dolfinx
except ImportError:
    !wget "https://github.com/fem-on-colab/fem-on-colab.github.io/raw/a3d664e/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

In [2]:
try:
    import viskex
except ImportError:
    !pip3 install "viskex@git+https://github.com/viskex/viskex.git@f26b326"
    import viskex

  Cloning https://github.com/viskex/viskex.git (to revision f26b326) to /private/var/folders/22/4jjz2l692gd5qg0yymb7mr1m0000gn/T/pip-install-sben99c4/viskex_937595f713ba46e9af85f8a38feb7cbf
  Running command git clone --filter=blob:none --quiet https://github.com/viskex/viskex.git /private/var/folders/22/4jjz2l692gd5qg0yymb7mr1m0000gn/T/pip-install-sben99c4/viskex_937595f713ba46e9af85f8a38feb7cbf
  Running command git checkout -q f26b326
  Resolved https://github.com/viskex/viskex.git to commit f26b326
  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached jupyter_server_proxy-4.4.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached trame-3.7.6-py3-none-any.whl.metadata (8.1 kB)
  Using cached trame_client-3.5.2-py3-none-any.whl.metadata (5.9 kB)
  Using cached trame_server-3.3.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached trame_vtk-2.8.14-py3-none-any.whl.metadata (13 kB)
  Using cached 

In [3]:
import dolfinx.fem
import dolfinx.fem.petsc
import dolfinx.mesh
import mpi4py.MPI
import numpy as np
import petsc4py.PETSc
import ufl
import viskex

# Tutorial 1: solving a diffusion problem in 1D

We consider the model boundary value problem:
$$
\left\{
\begin{array}{l}
- u'' = 2, & x \in I= (0, 1),\\
u(0) = 0,\\
u(1) = 1.
\end{array}
\right.
$$

**Task 1: create a mesh.**

`dolfinx.mesh` provide some built-in functions to generate simple meshes, and in particular `create_unit_interval` for an equispaced mesh on the unit interval $I$.

 Create the uniform mesh with 10 cells:

In [4]:
mesh = dolfinx.mesh.create_unit_interval(mpi4py.MPI.COMM_WORLD,12)

Note that `dolfinx.mesh` requires that we supply the MPI-communicator. This is to specify how we would like the program to behave in parallel. With:
* MPI.COMM_WORLD we create a single mesh, whose data is distributed over the number of processors we would like to use.
* MPI.COMM_SELF we create a separate mesh on each processor

We can obtain an interactive plot of the domain using viskex

In [5]:
viskex.dolfinx.plot_mesh(mesh)

Widget(value='<iframe src="http://localhost:59982/index.html?ui=P_0x16be3be00_0&reconnect=auto" class="pyvista…

A **mesh**  is made by
*  a set of points: these are part of the mesh.geometry
*  a set of subintervals that connect them: these are part of the mesh.topology



In [6]:
points = mesh.geometry.x
points



array([[0.        , 0.        , 0.        ],
       [0.08333333, 0.        , 0.        ],
       [0.16666667, 0.        , 0.        ],
       [0.25      , 0.        , 0.        ],
       [0.33333333, 0.        , 0.        ],
       [0.41666667, 0.        , 0.        ],
       [0.5       , 0.        , 0.        ],
       [0.58333333, 0.        , 0.        ],
       [0.66666667, 0.        , 0.        ],
       [0.75      , 0.        , 0.        ],
       [0.83333333, 0.        , 0.        ],
       [0.91666667, 0.        , 0.        ],
       [1.        , 0.        , 0.        ]])

In [7]:
connectivity_cells_to_vertices = mesh.topology.connectivity(mesh.topology.dim, 0)
connectivity_cells_to_vertices

<AdjacencyList> with 12 nodes
  0: [0 1 ]
  1: [1 2 ]
  2: [2 3 ]
  3: [3 4 ]
  4: [4 5 ]
  5: [5 6 ]
  6: [6 7 ]
  7: [7 8 ]
  8: [8 9 ]
  9: [9 10 ]
  10: [10 11 ]
  11: [11 12 ]

(Note that `dolfinx` developers decided to store points as vectors in $\mathbb{R}^3$, regardless of the actual ambient space dimension!)

Let's save the number of elements.

In [8]:
num_cells = len(connectivity_cells_to_vertices)
num_cells


12

We can have a look at each cell  by using a `for` loop. Each cell is assigned an unique ID and (in 1D) it is uniquely defined by two vertices, which correspond to the endpoints of the subinterval.

In [9]:
for c in range(num_cells):
    # Print the ID of the current cell
    print("Cell ID", c, "is defined by the following vertices:")
    # Print the vertices of the current cell
    for v in connectivity_cells_to_vertices.links(c):
        print("\t" + "Vertex ID", v, "is located at x =", points[v][0])

Cell ID 0 is defined by the following vertices:
	Vertex ID 0 is located at x = 0.0
	Vertex ID 1 is located at x = 0.08333333333333333
Cell ID 1 is defined by the following vertices:
	Vertex ID 1 is located at x = 0.08333333333333333
	Vertex ID 2 is located at x = 0.16666666666666666
Cell ID 2 is defined by the following vertices:
	Vertex ID 2 is located at x = 0.16666666666666666
	Vertex ID 3 is located at x = 0.25
Cell ID 3 is defined by the following vertices:
	Vertex ID 3 is located at x = 0.25
	Vertex ID 4 is located at x = 0.3333333333333333
Cell ID 4 is defined by the following vertices:
	Vertex ID 4 is located at x = 0.3333333333333333
	Vertex ID 5 is located at x = 0.41666666666666663
Cell ID 5 is defined by the following vertices:
	Vertex ID 5 is located at x = 0.41666666666666663
	Vertex ID 6 is located at x = 0.5
Cell ID 6 is defined by the following vertices:
	Vertex ID 6 is located at x = 0.5
	Vertex ID 7 is located at x = 0.5833333333333333
Cell ID 7 is defined by the fol

Next, we identify the IDs corresponding to boundary nodes. We use the

`dolfinx.mesh` function `locate_entities_boundary`. It requires the following inputs:
 * the first argument is the mesh,
 * the second argument represent the topological dimension of the mesh entities which we are interested in. In 1D, `mesh.topology.dim` is equal to 1, and entities of topological dimension 1 are the cells (subintervals), while `mesh.topology.dim - 1` is equal to 0, and entities of topological dimension 0 are the vertices of mesh.
 * the third argument is a condition (i.e., a function that returns either `True` or `False`) on the coordinates `x`, which are stored as a vector. Since we are interested in finding the vertex located at $x = 0$, we may think of using `x[0] == 0` as a condition: however, due to floating point arithmetic, it is safer to use $\left|x - 0\right| < \varepsilon$, where $\varepsilon$ is a small number, which may be written as `np.isclose(x[0], 0.0)`.

In [10]:
# Also the dimension is a topological info:
tdim = mesh.topology.dim
fdim = tdim - 1

left_boundary_entities = dolfinx.mesh.locate_entities_boundary(
    mesh, fdim, lambda x: np.isclose(x[0], 0.0))
left_boundary_entities

right_boundary_entities = dolfinx.mesh.locate_entities(
    mesh, fdim, lambda x: np.isclose(x[0], 1.0))
right_boundary_entities

array([12], dtype=int32)

**Task 2: create FEM space.**

We define the finite element function space $V_h$ using $\mathbb{P}_2$ Lagrange elements.

This is obtained using the `FunctionSpace` class of `dolfinx.fem`.

The first argument specifies the mesh. The second the type of FE space. To define the standard (conforming) Lagrange elements we input `"CG"`. Using instead `"Lagrange"` or `"P"` yields the same space.

In [11]:
Vh = dolfinx.fem.functionspace(mesh, ("Lagrange", 2))

Store the dimension of the space:

In [12]:
Vh_dim = Vh.dofmap.index_map.size_local
Vh_dim

25

Once the FE space is at hand, we introduce *ufl*  (unified form language) symbols to define the trial and test functions for our weak formulation:

In [13]:
uh = ufl.TrialFunction(Vh)
vh = ufl.TestFunction(Vh)

**Task 3:** Set up FEM system

Now we are ready to define the FEM using the ufl capability.
* `uh.dx(0)` corresponds to $\frac{\partial u}{\partial x}$, where the argument `0` to `dx` means to take the derivative with respect to the first space coordinate (the only one of interest in this case).
* `ufl.dx` provides a measure for integration over the domain. Integration will automatically occur over the entire domain.

In [14]:
dx = ufl.dx

A = uh.dx(0) * vh.dx(0) * dx

F = 2 * vh * dx

**Task 4:** Apply boundary conditions

It remains to implement the boundary conditions. To do so we:
* determine the degree of freedom that corresponds to the boundary vertices.
* define a `Constant` equal to `0` and a `Constant` equal to `1` corresponding to the values on the boundary.
* create a list containing the Dirichlet boundary conditions (two in this case), that is the constraints on the FE function DoF:

We can help ourselves looking at the following table, which has in the first colum the ID of the degree of freedom, and in the second column the corresponding 𝑥 coordinate.

In [15]:
with np.printoptions(precision=2, suppress=True):
    print(np.vstack((np.arange(Vh_dim), Vh.tabulate_dof_coordinates()[:, 0])).T)

[[ 0.   -0.  ]
 [ 1.    0.08]
 [ 2.    0.04]
 [ 3.    0.17]
 [ 4.    0.12]
 [ 5.    0.25]
 [ 6.    0.21]
 [ 7.    0.33]
 [ 8.    0.29]
 [ 9.    0.42]
 [10.    0.37]
 [11.    0.5 ]
 [12.    0.46]
 [13.    0.58]
 [14.    0.54]
 [15.    0.67]
 [16.    0.62]
 [17.    0.75]
 [18.    0.71]
 [19.    0.83]
 [20.    0.79]
 [21.    0.92]
 [22.    0.87]
 [23.    1.  ]
 [24.    0.96]]


In [16]:
left_boundary_dofs = dolfinx.fem.locate_dofs_topological(Vh, mesh.topology.dim-1, left_boundary_entities)

In [17]:
right_boundary_dofs = dolfinx.fem.locate_dofs_topological(Vh, mesh.topology.dim-1, right_boundary_entities)

In [18]:
zero = dolfinx.fem.Constant(mesh, 0.)
one = dolfinx.fem.Constant(mesh, 1.)

In [19]:
bcs = [dolfinx.fem.dirichletbc(zero, left_boundary_dofs, Vh), dolfinx.fem.dirichletbc(one, right_boundary_dofs, Vh)]

**Task 5:** Solve the FEM system

In order to solve the FEM system, we go through the following steps:

* `dolfinx.fem` provides a `Function` class to store the solution of a finite element problem:
* solve the discrete problem allocating a new `LinearProblem` (which uses `PETSc`), providing as input the bilinear form `a`, the linear functional `F`, the boundary conditions `bcs`, and where to store the solution. Further solver options can also be passed to `PETSc`.

In [20]:
solution = dolfinx.fem.Function(Vh)

In [21]:
problem = dolfinx.fem.petsc.LinearProblem(
    A, F, bcs=bcs, u=solution,
    petsc_options={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
_ = problem.solve()

Here are the computed FEM solution's DoF:

In [22]:
solution_vector = solution.x.array
print(solution_vector)

[0.         0.15972222 0.08159722 0.30555556 0.234375   0.4375
 0.37326389 0.55555556 0.49826389 0.65972222 0.609375   0.75
 0.70659722 0.82638889 0.78993056 0.88888889 0.859375   0.9375
 0.91493056 0.97222222 0.95659722 0.99305556 0.984375   1.
 0.99826389]


...  and corresponding plot of the solution

In [23]:
viskex.dolfinx.plot_scalar_field(solution, "u_h")

Widget(value='<iframe src="http://localhost:59982/index.html?ui=P_0x16c65fed0_1&reconnect=auto" class="pyvista…

In [24]:
viskex.dolfinx.plot_scalar_field(solution, "u_h", warp_factor=1.0)

Widget(value='<iframe src="http://localhost:59982/index.html?ui=P_0x16c8d1310_2&reconnect=auto" class="pyvista…

# Compute error

**Task 6:** compute the $L^2$ and $H^1$ errors.

The exact solution is:
$$ u(x) = - x^2 + 2 x.$$

The $L^2(I)$ norm of the error $u - u_h$ is defined as:
$$ e_h^2 = \int_I \left(u(x) - u_h(x)\right)^2 \ \mathrm{d}x.$$

In order to evaluate the error, we first need to define a symbolic representation in `ufl` of the exact solution $u(x)$. To this end, we need to define a symbol for the coordinate `x` ...

In [25]:
xyz = ufl.SpatialCoordinate(mesh)
x = xyz[0]

and then we can define a symbolic expression in `ufl` for the exact solution $u$:

In [26]:
exact_solution = - x**2 + 2 * x

Hence we can define a symbolic expression in `ufl` for the integral defining the error:

In [27]:
error_L2squared_ufl = (exact_solution - solution)**2 * dx

Finally, we evaluate the error using the `dolfinx.fem` function `assemble_scalar`:

In [28]:
error_L2squared = dolfinx.fem.assemble_scalar(dolfinx.fem.form(error_L2squared_ufl))
print(error_L2squared)

4.348237057869541e-29


Note that, given that we are using quadratic elements, we expect the error to be zero!

Similarly we can compute the H1 seminorm error:

In [29]:
error_H1squared_ufl = (exact_solution.dx(0) - solution.dx(0) ) **2 * dx
error_H1squared = dolfinx.fem.assemble_scalar(dolfinx.fem.form(error_H1squared_ufl))
error_H1squared

4.519613845317649e-28